In [12]:
import heapq
import random
import numpy as np
from timeit import default_timer as timer
from datetime import timedelta
from time import process_time
import math
import time

In [13]:
#PROBLEM
#AIMA CODE
class Problem(object):
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal
  
    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        return state.append(action)

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if (len(state) ==self.goal+1) and state[len(state)-1] == state[0]:
            return True
        else:
            return False

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        return 0



In [14]:
#NODE
#AIMA CODE
class Node:

    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state.  Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node.  Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action,
                    problem.path_cost(self.path_cost, self.state,
                                      action, next_state))
        return next_node
    
    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        return hash(self.state)

In [15]:
#QUEUE
#AIMA CODE
class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []

        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)

#MEMOIZE
def memoize(fn, slot=None, maxsize=32):
    """Memoize fn: make it remember the computed value for any argument list.
    If slot is specified, store result in that slot of first argument.
    If slot is false, use lru_cache for caching the values."""
    if slot:
        def memoized_fn(obj, *args):
            if hasattr(obj, slot):
                return getattr(obj, slot)
            else:
                val = fn(obj, *args)
                setattr(obj, slot, val)
                return val
    else:
        @functools.lru_cache(maxsize=maxsize)
        def memoized_fn(*args):
            return fn(*args)

    return memoized_fn

In [16]:
#GRAPH
from collections import defaultdict 

#from geeksforgeeks primm MST algorithm:https://www.geeksforgeeks.org/prims-minimum-spanning-tree-mst-greedy-algo-5/
class PrimGraph():

    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]

    # A utility function to print the constructed MST stored in parent[]
    def printMST(self, parent):
        print
        "Edge \tWeight"
        for i in range(1, self.V):
            print
            parent[i], "-", i, "\t", self.graph[i][parent[i]]

    def add_cost(self,parent):
        cost = 0
        for i in range(1, self.V):
            cost = cost + self.graph[i][parent[i]]
        return cost

    # A utility function to find the vertex with
    # minimum distance value, from the set of vertices
    # not yet included in shortest path tree
    def minKey(self, key, mstSet):

        # Initilaize min value
        min = math.inf

        for v in range(self.V):
            if key[v] < min and mstSet[v] == False:
                min = key[v]
                min_index = v

        return min_index

    # Function to construct and print MST for a graph
    # represented using adjacency matrix representation
    def primMST(self):

        # Key values used to pick minimum weight edge in cut
        key = [math.inf] * self.V
        parent = [None] * self.V  # Array to store constructed MST
        # Make key 0 so that this vertex is picked as first vertex
        key[0] = 0
        mstSet = [False] * self.V

        parent[0] = -1  # First node is always the root of

        for cout in range(self.V):

            # Pick the minimum distance vertex from
            # the set of vertices not yet processed.
            # u is always equal to src in first iteration
            u = self.minKey(key, mstSet)

            # Put the minimum distance vertex in
            # the shortest path tree
            mstSet[u] = True

            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shotest path tree
            for v in range(self.V):

                # graph[u][v] is non zero only for adjacent vertices of m
                # mstSet[v] is false for vertices not yet included in MST
                # Update the key only if graph[u][v] is smaller than key[v]
                if self.graph[u][v] > 0 and mstSet[v] == False and key[v] > self.graph[u][v]:
                    key[v] = self.graph[u][v]
                    parent[v] = u

        return self.add_cost(parent)

#from geeksforgeeks kruskal MST algorithm:https://www.geeksforgeeks.org/kruskals-minimum-spanning-tree-algorithm-greedy-algo-2/
class MSTGraph():
    def __init__(self, vertices):
        self.V = vertices  # No. of vertices
        self.graph = []  # default dictionary
        # to store graph

    # function to add an edge to graph
    def addEdge(self, u, v, w):
        self.graph.append([u, v, w])

        # A utility function to find set of an element i

    # (uses path compression technique)
    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

        # A function that does union of two sets of x and y

    # (uses union by rank)
    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)

        # Attach smaller rank tree under root of
        # high rank tree (Union by Rank)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot

            # If ranks are same, then make one as root
        # and increment its rank by one
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    # The main function to construct MST using Kruskal's
    # algorithm
    def KruskalMST(self):

        result = []  # This will store the resultant MST
        i = 0  # An index variable, used for sorted edges
        e = 0  # An index variable, used for result[]
       # print(self.graph)
        # Step 1:  Sort all the edges in non-decreasing
        # order of their
        # weight.  If we are not allowed to change the
        # given graph, we can create a copy of graph
        self.graph = sorted(self.graph, key=lambda item: item[2])
        parent = [];
        rank = []

        # Create V subsets with single elements
        for node in range(self.V):
            parent.append(node)
            rank.append(0)

            # Number of edges to be taken is equal to V-1
        while e < self.V - 1:

            # Step 2: Pick the smallest edge and increment
            # the index for next iteration

            u, v, w = self.graph[i]

            i = i + 1

            x = self.find(parent, u)
            y = self.find(parent, v)


            # If including this edge does't cause cycle,
            # include it in result and increment the index
            # of result for next edge
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)

        return result

#AIMA CODE
class Graph:
    """A graph connects nodes (vertices) by edges (links). Each edge can also
    have a length associated with it. The constructor call is something like:
        g = Graph({'A': {'B': 1, 'C': 2})
    this makes a graph with 3 nodes, A, B, and C, with an edge of length 1 from
    A to B,  and an edge of length 2 from A to C. You can also do:
        g = Graph({'A': {'B': 1, 'C': 2}, directed=False)
    This makes an undirected graph, so inverse links are also added. The graph
    stays undirected; if you add more links with g.connect('B', 'C', 3), then
    inverse link is also added. You can use g.nodes() to get a list of nodes,
    g.get('A') to get a dict of links out of A, and g.get('A', 'B') to get the
    length of the link from A to B. 'Lengths' can actually be any object at
    all, and nodes can be any hashable object."""

    def __init__(self, graph_dict=None, directed=True):
        self.graph_dict = graph_dict or {}
        self.directed = directed
        if not directed:
            self.make_undirected()

    def make_undirected(self):
        """Make a digraph into an undirected graph by adding symmetric edges."""
        for a in list(self.graph_dict.keys()):
            for (b, dist) in self.graph_dict[a].items():
                self.connect1(b, a, dist)

    def connect(self, A, B, distance=1):
        """Add a link from A and B of given distance, and also add the inverse
        link if the graph is undirected."""
        self.connect1(A, B, distance)
        if not self.directed:
            self.connect1(B, A, distance)

    def connect1(self, A, B, distance):
        """Add a link from A to B of given distance, in one direction only."""
        self.graph_dict.setdefault(A, {})[B] = distance

    def get(self, a, b=None):
        """Return a link distance or a dict of {node: distance} entries.
        .get(a,b) returns the distance or None;
        .get(a) returns a dict of {node: distance} entries, possibly {}."""
        links = self.graph_dict.setdefault(a, {})
        if b is None:
            return links
        else:
            return links.get(b)

    def nodes(self):
        """Return a list of nodes in the graph."""
        s1 = set([k for k in self.graph_dict.keys()])
        s2 = set([k2 for v in self.graph_dict.values() for k2, v2 in v.items()])
        nodes = s1.union(s2)
        return list(nodes)


def UndirectedGraph(graph_dict=None):
    """Build a Graph where every edge (including future ones) goes both ways."""
    return Graph(graph_dict, directed=False)

#AIMA CODE
class GraphProblem(Problem):

    """The problem of searching a graph from one node to another."""

    def __init__(self, initial, goal, graph,ajc):
        Problem.__init__(self, initial, goal)
        self.graph = graph
        self.visited=[[]]
        self.adjmatrix=ajc

    def actions(self, A):
        """The actions at a graph node are just its neighbors."""
        fin_lst=[]
        #print(A)
        templst=list(self.graph.get(A[len(A)-1]).keys())
        if len(A)==self.goal:
            b=list(A)
            b.append(self.initial)
            fin_lst.append(b)
        for i in templst:
          if i not in A:
            b=list(A)
            b.append(i)
            fin_lst.append(b)
            #print(fin_lst)
        return fin_lst

    def result(self, state, action):
        """The result of going to a neighbor is just that neighbor."""
        return action

    def path_cost(self, cost_so_far, A, action, B):
      length=len(B)
      return cost_so_far + (self.graph.get(B[length-2], B[length-1]))

    def find_min_edge(self):
        """Find minimum value of edges."""
        m = np.Inf
        for d in self.graph.graph_dict.values():
            local_min = min(d.values())
            m = min(m, local_min)

        return m

    def h(self, node):
        """h function is straight-line distance from a node's state to goal."""
        locs = getattr(self.graph, 'locations', None)
        if locs:
            if type(node) is str:
                return int(distance(locs[node], locs[self.goal]))

            return int(distance(locs[node.state], locs[self.goal]))
        else:
            return np.Inf

    def cheapest_edges(self,A):
      cost = 0
      length=len(A.state)
      ints=[]
      if length != 1:
        for character in A.state:
          ints.append(ord(character)-65)

      for i in range(self.goal):
        if i not in ints:
          templist=list(self.adjmatrix[i])
          templist.sort()
          cost += templist[1]
      return cost

    def random_edges(self, A):
            cost = 0
            length = len(A.state)
            ints = []

            if length == self.goal:
                return self.graph.get(A.state[length-1], self.initial)

            if length != 1:
                for character in A.state:
                    ints.append(ord(character) - 65)

            for i in range(self.goal):
                if i not in ints:
                    index = random.randrange(0, self.goal)
                    while index not in ints and index != i:
                        index = random.randrange(0, self.goal)
                    cost = cost + self.adjmatrix[i][index]
            return cost

    def prim_mst_heuristic(self, A):
        cost = 0
        g = PrimGraph(self.goal - len(A.state))
        ints = []
        for character in A.state:
            ints.append(ord(character) - 65)

        adjmatrix = []
        if len(A.state) == self.goal:
            return self.graph.get(A.state[len(A.state) - 1], self.initial)
        if len(A.state) > self.goal:
            return 0

        for x in range(self.goal):
            templist = []
            if x not in ints:
                for y in range(self.goal):
                    if y not in ints:
                        templist.append(self.adjmatrix[x][y])
            if x not in ints:
                adjmatrix.append(templist)

        g.graph = adjmatrix
        cost = g.primMST()
        return cost

    def mst_heuristic(self,A):
        cost=0
        g=MSTGraph(self.goal-len(A.state))
        ints=[]
        for character in A.state:
          ints.append(ord(character)-65)
          newadjmatrix=[]
          for index in range(self.goal):
            if index not in ints:
              newadjmatrix.append(self.adjmatrix[index])
          newnewadjmatrix=[]
          for v1 in range(len(newadjmatrix)):
            templist=[] 
            for v2 in range(self.goal):
              if v2 not in ints and v1 != v2:
                templist.append(self.adjmatrix[v1][v2])
            newnewadjmatrix.append(templist)

        for x in range(len(newnewadjmatrix)):
          for y in range(len(newnewadjmatrix[x])):
            g.addEdge(x, y, newnewadjmatrix[x][y])
        result=g.KruskalMST()
        for v1, v2, weight in result:
          cost += weight
      
        return cost


In [17]:
#SEARCH
#AIMA CODE
def best_first_graph_search(problem, f, display=False):
    """Search the nodes with the lowest f scores first.
    You specify the function f(node) that you want to minimize; for example,
    if f is a heuristic estimate to the goal, then we have greedy best
    first search; if f is node.depth then we have breadth-first search.
    There is a subtlety: the line "f = memoize(f, 'f')" means that the f
    values will be cached on the nodes as they are computed. So after doing
    a best first search you can examine the f values of the path returned."""
    f = memoize(f, 'f')
    node = Node(problem.initial)
    frontier = PriorityQueue('min', f)
    frontier.append(node)
    global total_nodes
    total_nodes=0
    explored = set()
    while frontier:
        node = frontier.pop()
        #print(node.path_cost)
        total_nodes += 1
        if problem.goal_test(node.state):
        #if len(node.path()) ==3:
            return node
        #print(node.state)
        if tuple(node.state) not in explored:
          explored.add(tuple(node.state))
        
        for child in node.expand(problem):
            if tuple(child.state) not in explored and child not in frontier:
                frontier.append(child)
            elif child in frontier:
              #print(child)#is this where to prune
              if f(child) < frontier[child]:
                  del frontier[child]
                  path.append(child)
    return None

#ASTAR
def astar_search(problem, h=None, display=False):
    """A* search is best-first graph search with f(n) = g(n)+h(n).
    You need to specify the h function when you call astar_search, or
    else in your Problem subclass."""
    h = memoize(h or problem.h, 'h')
    return best_first_graph_search(problem, lambda n: n.path_cost + h(n), display)


def uniform_cost_search(problem, display=False):
    """[Figure 3.14]"""
    return best_first_graph_search(problem, lambda node: node.path_cost, display)



In [18]:
#HILL CLIMBING
#AIMA CODE

identity = lambda x: x
def hill_climbing(problem):
    
    """From the initial node, keep choosing the neighbor with highest value,
    stopping when no neighbor is better. [Figure 4.2]"""
    
    def find_neighbors(state, number_of_neighbors=10):
        """ finds neighbors using two_opt method """
        
        neighbors = []
        
        for i in range(number_of_neighbors):
            new_state = problem.two_opt(state)
            neighbors.append(Node(new_state))
            state = new_state
            
        return neighbors

    # as this is a stochastic algorithm, we will set a cap on the number of iterations
    iterations = 10000
    
    current = Node(problem.initial)
    #print(current)
    while iterations:
        neighbors = find_neighbors(current.state)
        if not neighbors:
            break
        neighbor = argmax_random_tie(neighbors,
                                     key=lambda node: problem.value(node.state))
        if problem.value(neighbor.state) <= problem.value(current.state):
           """Note that it is based on negative path cost method"""
           current.state = neighbor.state
        iterations -= 1
        
    return current.state

def argmin_random_tie(seq, key=identity):
    """Return a minimum element of seq; break ties at random."""
    return min(shuffled(seq), key=key)


def argmax_random_tie(seq, key=identity):
    """Return an element with highest fn(seq[i]) score; break ties at random."""
    return max(shuffled(seq), key=key)


def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items
#AIMA CODE
class TSP_problem(Problem):

    """ subclass of Problem to define various functions """
    def __init__(self, initial, dist):
        self.initial = initial
        self.distances = dist

    def two_opt(self, state):
        """ Neighbour generating function for Traveling Salesman Problem """
        neighbour_state = state[:]
        left = random.randint(0, len(neighbour_state) - 1)
        right = random.randint(0, len(neighbour_state) - 1)
        if left > right:
            left, right = right, left
        neighbour_state[left: right + 1] = reversed(neighbour_state[left: right + 1])
        return neighbour_state

    def actions(self, state):
        """ action that can be excuted in given state """
        return [self.two_opt]

    def result(self, state, action):
        """  result after applying the given action on the given state """
        return action(state)

    def path_cost(self, c, state1, action, state2):
        """ total distance for the Traveling Salesman to be covered if in state2  """
        cost = 0
        for i in range(len(state2) - 1):
            cost += self.distances[state2[i]][state2[i + 1]]
        cost += self.distances[state2[0]][state2[-1]]
        return cost

    def value(self, state):
        """ value of path cost given negative for the given state """
        return -1 * self.path_cost(None, None, None, state)

In [19]:
#RANDOM GRAPH
import os
import numpy as np

#REGLI CODE 
def create_final_output(graph, v_num, x):
	fo_new = open("./graphs/infile%02d_%02d.txt" % (v_num, x+1), "w+")
	fo_new.write(str(v_num) + "\n")
	z = 0
	while(z < v_num):
		temp_s = ""
		i = 0
		while(i < v_num):
			if(i == 0):
				temp_s = temp_s + str(int(graph[z, i]))
			else:
				temp_s = temp_s + ", " + str(int(graph[z, i]))
			i += 1
		temp_s = temp_s + '\n'
		fo_new.write(temp_s)
		z += 1
	fo_new.close()

def create_complete_symmetric_graphs(n):
	points = []
	i = 0
	while(i < n):
		#make point on unit square for each vertex
		point = [np.random.randint(1000), np.random.randint(1000)]
		points.append(point)
		i += 1

	euclidian_graph = np.zeros((n, n))

	i = 0
	while(i < n):
		ii = 0
		while(ii < n): #doing this the dumb and fast way
			euclidian_graph[i,ii] = int(np.sqrt(((points[i][0]-points[ii][0])*(points[i][0]-points[ii][0])) + ((points[i][1]-points[ii][1])*(points[i][1]-points[ii][1]))))
			ii += 1
		i += 1

	return euclidian_graph

def eat_plantri_output(fname, x_temp, graphs):
	fo = open(fname, "r")
	x = 0 + x_temp
	v_num = 0

	for line in fo:

		t0 = line.split(" ")
		t1 = (t0[1].replace('\n', '')).split(",")

		v_num = int(t0[0])
		#print(v_num)

		points = []
		i = 0
		while(i < v_num):
			#make point on unit square for each vertex
			point = [np.random.randint(1000), np.random.randint(1000)]
			points.append(point)
			i += 1

		euclidian_graph = np.zeros((v_num, v_num))

		i = 0
		while(i < v_num):
			ii = 0
			while(ii < v_num): #doing this the dumb and fast way
				euclidian_graph[i,ii] = np.sqrt(((points[i][0]-points[ii][0])*(points[i][0]-points[ii][0])) + ((points[i][1]-points[ii][1])*(points[i][1]-points[ii][1])))
				ii += 1
			i += 1

		graph = np.zeros((v_num, v_num))
		z = 0
		while(z < v_num):
			for c in t1[z]:
				#print(ord(c) - 97)
				if(graph[z, ord(c) - 97] == 0):
					graph[z, ord(c) - 97] = 1
					graph[z, ord(c) - 97] = euclidian_graph[z, ord(c) - 97]
					#add connection, set to distance in euclidian graph
			#print(t1[z])
			z += 1
		graphs.append(graph)
		x += 1
	#print(x)
	fo.close()
	if(x < 30):
		eat_plantri_output(fname, x, graphs)
		#recurse until we have 30 to make up the five missing for the 5-node example
	else:
		#create the final outputs
		picks = []
		pick = 0
		i = 0
		while(i < 30):
			pick = np.random.randint(x) #pick a random graph from our selection
			while(pick in picks):
				pick = np.random.randint(x)
			create_final_output(graphs[pick], v_num, i)
			picks.append(pick)
			i += 1


In [33]:
def randomGraph(n):
  dict_numbers_to_letters = {}
  for x in range(26):
    dict_numbers_to_letters[x] = chr(x + 65)
  dict_numbers_to_letters[26] = 'AA'
  dict_numbers_to_letters[27] = 'BB'
  dict_numbers_to_letters[28] = 'CC'
  dict_numbers_to_letters[29] = 'DD'
 
  f=create_complete_symmetric_graphs(n)
  adj=f
  d=dict()
  for row in range(n):
    d[chr(row+65)]=dict()
    for col in range(n):
      if adj[row][col] !=0:
        d[chr(row+65)][chr(col+65)]=adj[row][col]
  return (UndirectedGraph(d),d,adj)

def calculate_cst(amnt, sol,graph_d):
  cst = 0
  for i in range(amnt):
    cst += graph_d[sol[i]][sol[i + 1]]
  return (cst)

def search_data(prob,adj,n):
  lst=[]
  cheapest=prob.cheapest_edges
  rand=prob.random_edges
  # start = timer()
  # t1_start = process_time()  
  # final_path_astar=astar_search(prob).solution()[n-1]
  # t1_stop = process_time() 
  # end = timer()
  # cpu=timedelta(seconds=t1_stop-t1_start)
  # wall=timedelta(seconds=end-start)
  # cost=calculate_cst(n, final_path_astar,adj)
  # lst.append(("ASTAR NORMAL",final_path_astar,total_nodes,cost,cpu,wall))

  start = timer()
  t1_start = process_time()  
  final_path_astar=astar_search(prob,rand).solution()[n-1]
  t1_stop = process_time() 
  end = timer()
  cpu=timedelta(seconds=t1_stop-t1_start)
  wall=timedelta(seconds=end-start)
  cost=calculate_cst(n, final_path_astar,adj)
  lst.append(("ASTAR RANDOM",final_path_astar,total_nodes,cost,cpu,wall))
  
  start = timer()
  t1_start = process_time()  
  final_path_astar=astar_search(prob,cheapest).solution()[n-1]
  t1_stop = process_time() 
  end = timer()
  cpu=timedelta(seconds=t1_stop-t1_start)
  wall=timedelta(seconds=end-start)
  cost=calculate_cst(n, final_path_astar,adj)
  lst.append(("ASTAR CHEAPEST",final_path_astar,total_nodes,cost,cpu,wall))
  
  tart = timer()
  t1_start = process_time()  
  final_path_astar=uniform_cost_search(prob).solution()[n-1]
  t1_stop = process_time() 
  end = timer()
  cpu=timedelta(seconds=t1_stop-t1_start)
  wall=timedelta(seconds=end-start)
  cost=calculate_cst(n, final_path_astar,adj)
  lst.append(("UCS",final_path_astar,total_nodes,cost,cpu,wall))
  
  # distances = adj
  # all_cities = []

  # for city in adj.keys():
  #  all_cities.append(city)
  # all_cities.sort()

  # tsp=TSP_problem(all_cities,distances)
  # start = timer()
  # t1_start = process_time()  
  # final_path_hc=hill_climbing(tsp)
  # t1_stop = process_time() 
  # end = timer()
  # cpu=timedelta(seconds=t1_stop-t1_start)
  # wall=timedelta(seconds=end-start)
  # cost=calculate_cst(n, final_path_astar,adj)
  # lst.append(("HC",final_path_hc,total_nodes,cost,cpu,wall))
  
  # start = timer()
  # t1_start = process_time()
  # mst=problem.mst_heuristic
  # final_path_astar=astar_search(prob,mst).solution()
  # t1_stop = process_time() 
  # end = timer()
  # cpu=timedelta(seconds=t1_stop-t1_start)
  # wall=timedelta(seconds=end-start)
  # cost=calculate_cst(n, final_path_astar,adj)
  # lst.append(("ASTAR MST",final_path_astar,total_nodes,cost,cpu,wall))


  start = timer()
  t1_start = process_time()
  pmst=prob.prim_mst_heuristic
  final_path_astar=astar_search(prob,pmst).solution()[n-1]
  t1_stop = process_time() 
  end = timer()
  cpu=timedelta(seconds=t1_stop-t1_start)
  wall=timedelta(seconds=end-start)
  cost=calculate_cst(n, final_path_astar,adj)
  lst.append(("ASTAR PRIMM MST",final_path_astar,total_nodes,cost,cpu,wall))

  return  lst

def batch(n,num_graphs):
  matrix=[]
  final_arr=[]
  for i in range(num_graphs):
    q=randomGraph(n)
    matrix.append(q[2])
    rand_start=chr(random.randrange(n)+65)
    problem = GraphProblem(rand_start, n, q[0], q[2])
    final_arr.append(search_data(problem,q[1],n))
  return final_arr,matrix

# n=10
# y=randomGraph(n)
# #print(y)
# rand_start=chr(random.randrange(n)+65)
# #print(rand_start) 
# problem = GraphProblem(rand_start, n, y[0], y[2])


n=10
num_graphs=30

# res=batch(n,num_graphs)
# for i in range(num_graphs):
#   print(res[0][i])
#   print("\n")

from google.colab import drive
drive.mount('/content/drive')

# with open('/content/drive/My Drive/Sandbox_2.json', 'w') as f:
#   for item in res:
#     f.write("%s\n" % item)

Mounted at /content/drive


In [21]:
import sys
distances = {}
all_cities = []

n=10
global numberOfVertices
numberOfVertices=10
y=randomGraph(n)
#print(y)
rand_start=chr(random.randrange(n)+65)
#print(rand_start) 
problem = GraphProblem(rand_start, n, y[0], y[2])


for city in y[1].keys():
    distances[city] = {}
    all_cities.append(city)
    
all_cities.sort()
distances = y[1]

tsp = TSP_problem(all_cities, distances)

#AIMA CODE
def exp_schedule(k=20, lam=0.005, limit=100):
    """One possible schedule function for simulated annealing"""
    return lambda t: (k * math.exp(-lam * t) if t < limit else 0)

def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

def simulated_annealing(problem, schedule=exp_schedule()):
    """[Figure 4.5] CAUTION: This differs from the pseudocode as it
    returns a state instead of a Node."""
    current = Node(problem.initial)
    for t in range(sys.maxsize):
        T = schedule(t)
        if T == 0:
            return current.state
        neighbors = current.expand(problem)
        if not neighbors:
            return current.state
        next_choice = random.choice(neighbors)
        delta_e = problem.value(next_choice.state) - problem.value(current.state)
        if delta_e > 0 or probability(math.exp(delta_e / T)):
            current = next_choice

#need to run this 100 times and then take the min path

def calc_cost(soln, graph):
    cost = 0
    for i in range(len(soln)-1):
        cost += graph.get(soln[i],soln[i+1])
    return cost


In [75]:
#AIMA CODE
import bisect
def weighted_sampler(seq, weights):
  totals = []
  for w in weights:
    totals.append(w + totals[-1] if totals else w)
        
  return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]


class GeneticAlgorithm:

    def __init__(self, numberOfVertices, adjmatrix, numbers_to_letters_dict):
        self.numberOfVertices = numberOfVertices
        self.adjmatrix = adjmatrix
        self.dict_numbers_to_letters = numbers_to_letters_dict


        """  def genetic_search(problem, ngen=1000, pmut=0.1, n=20):
        Call genetic_algorithm on the appropriate parts of a problem.
        This requires the problem to have states that can mate and mutate,
        plus a value method that scores states.

        # NOTE: This is not tested and might not work.
        # TODO: Use this function to make Problems work with genetic_algorithm.

        s = problem.initial_state
        states = [problem.result(s, a) for a in problem.actions(s)]
        random.shuffle(states)
        return genetic_algorithm(states[:n], problem.value, ngen, pmut)
        """
 

    def calc_cost(self, path):
        cost = 0
        for i in range(len(path) - 1):
            cost += self.adjmatrix[path[i]][path[i + 1]]
        return cost

    def fitness_fn(self, population):
        path_ints = []
        for x in range(len(population)):
          if population[x]=='AA':
            path_ints.append(26)
          elif population[x]=='BB':
            path_ints.append(27)
          elif population[x]=='CC':
            path_ints.append(28)
          elif population[x]=='DD':
            path_ints.append(29)
          else:
            path_ints.append(ord(population[x]) - 65)
          #path_ints.append(ord(population[x]) - 65)
        return 31000 - (self.calc_cost(path_ints))


    def init_population(self, pop_number, state_length):
        """Initializes population for genetic algorithm
        pop_number  :  Number of individuals in population
        gene_pool   :  List of possible values for individuals
        state_length:  The length of each individual"""
        g = state_length
        population = []
        original_lst = []

        for i in range(self.numberOfVertices):
            if i == 26:
                original_lst.append("AA")
            elif i == 27:
                original_lst.append("BB")
            elif i == 28:
                original_lst.append("CC")
            elif i == 29:
                original_lst.append("DD")
            elif i == 30:
                original_lst.append("EE")
            else:
                original_lst.append(chr(i + 65))

        for n in range(pop_number):
            random.shuffle(original_lst)
            population.append(original_lst.copy())


        return population

    def fitness_threshold(self, fitness_fn, f_thres, population):
        if not f_thres:
            return None

        fittest_individual = max(population, key=self.fitness_fn)
        if self.fitness_fn(fittest_individual) >= f_thres:
            return fittest_individual

        return None


    def select(self, r, population, fitness_fn):
        fitnesses = map(self.fitness_fn, population)
        sampler = weighted_sampler(population, fitnesses)
        return [sampler() for i in range(r)]


    def recombine(self, x, y):
        n = len(x)
        c = random.randrange(0, n)
        return x[:c] + y[c:]


    def recombine_uniform(self, x, y):
        n = len(x)
        result = [0] * n
        indexes = random.sample(range(n), n)
        for i in range(n):
            ix = indexes[i]
            result[ix] = x[ix] if i < n / 2 else y[ix]

        return ''.join(str(r) for r in result)

    def recombine_ordered(self, x, y):
        n = len(x)
        first = random.randrange(0, n-2)
        second = random.randrange(first, n)
        temp = []
        child = []
        lst = x[first:second]
        for i in lst:
            y.remove(i)

        y[second:second] = lst
        return y



    def mutate(self, x, gene_pool, pmut):
        if random.uniform(0, 1) >= pmut:
            return x

        n = len(x)
        g = len(gene_pool)
        c = random.randrange(0, n)
        r = random.randrange(c, n)

        first_gene = x[c]
        second_gene = x[r]
        templist = []
        for i in range(len(x)):
            if i == c:
                templist.append(second_gene)
            elif i == r:
                templist.append(first_gene)
            else:
                templist.append(x[i])
        #print(templist)
        return templist

    

    def genetic_algorithm(self, population, fitness_fn, gene_pool, f_thres=None, ngen=10000, pmut=0.1):
        """[Figure 4.8]"""
        for i in range(ngen):
            population = [self.mutate(self.recombine_ordered(*self.select(2, population, fitness_fn)), gene_pool, pmut)
                          for i in range(len(population))]
            #print(population)
            fittest_individual = self.fitness_threshold(fitness_fn, f_thres, population)
            if fittest_individual:
                return fittest_individual

        return max(population, key=self.fitness_fn)

In [76]:
dict_numbers_to_letters = {}
for x in range(26):
  dict_numbers_to_letters[x] = chr(x + 65)
dict_numbers_to_letters[26] = 'AA'
dict_numbers_to_letters[27] = 'BB'
dict_numbers_to_letters[28] = 'CC'
dict_numbers_to_letters[29] = 'DD'
n=30
global numberOfVertices
numberOfVertices=30
y=randomGraph(n)
#print(y)
rand_start=chr(random.randrange(n)+65)
#print(rand_start) 
problem = GraphProblem(rand_start, n, y[0], y[2])
#print(y[2])


g=GeneticAlgorithm(numberOfVertices, y[2], dict_numbers_to_letters)
genetic_initial_pop=g.init_population(100,numberOfVertices)
genetic_fitness=g.fitness_fn
gene_pool=[]
for x in range(numberOfVertices):
  gene_pool.append(dict_numbers_to_letters[x])
sol=g.genetic_algorithm(genetic_initial_pop,genetic_fitness,gene_pool, 100)
gensol=sol
gensol.append(gensol[0])

print(gensol)

['G', 'Z', 'AA', 'U', 'X', 'L', 'BB', 'I', 'F', 'T', 'C', 'V', 'B', 'P', 'D', 'S', 'DD', 'R', 'M', 'N', 'J', 'A', 'Y', 'Q', 'CC', 'H', 'E', 'W', 'O', 'K', 'G']


In [90]:

fields = ['Size', 'Min Cost', 'Avg Cost', 'Max Cost', 'Min Nodes Expanded', 'Avg Nodes Expanded', 'Max Nodes Expanded',
              'Min Real Time', 'Avg Real Time','Max Real Time','Min CPU Time','Avg CPU Time','Max CPU Time']

ucs_stats = []
random_edges_stats = []
cheapest_edges_stats = []
mst_stats = []
hc_stats = []
sa_stats = []
ga_stats = []

ucs_cost = []
ucs_realtime = []
ucs_cpu_time = []
ucs_num_nodes = []

rand_cost = []
rand_realtime = []
rand_cpu_time = []
rand_num_nodes = []

cheapest_cost = []
cheapest_realtime = []
cheapest_cpu_time = []
cheapest_num_nodes = []

mst_cost = []
mst_realtime = []
mst_cpu_time = []
mst_num_nodes = []

hc_cost = []
hc_realtime = []
hc_cpu_time = []
hc_num_nodes = []

sa_cost = []
sa_realtime = []
sa_cpu_time = []
sa_num_nodes = []

ga_cost = []
ga_realtime = []
ga_cpu_time = []
ga_num_nodes = []

def calc_cost(soln, graph):
    cost = 0
    for i in range(len(soln)-1):
      if graph.get(soln[i],soln[i+1]) != None:
        cost += graph.get(soln[i],soln[i+1])  
    return cost

for i in range(30):
  size=8
  all_cities=[]
  y=randomGraph(size)
  graphMatrix = y[2]
  #print(graphMatrix)
  graphDict = y[1]
  #print(graphDict)
  graph = y[0]
  #Generate random starting node
  start = chr(random.randrange(0,size) + 65)

  for city in graphDict.keys():
    if city not in all_cities:
      all_cities.append(city)

  #Create the problem instance
  prob = GraphProblem(start,size, graph, graphMatrix)

  ucs_heuristic = prob.h(0)
  random_edges_heuristic = prob.random_edges
  cheapest_edges_heuristic = prob.cheapest_edges
  mst_heuristic = prob.prim_mst_heuristic

  # start_time = time.time()
  # start_process_time = time.process_time()
  # soln1 = uniform_cost_search(prob).solution()[size - 1]
  # elapsed_process_time = time.process_time() - start_process_time
  # elapsed_time = time.time() - start_time

  # ucs_cost.append(calc_cost(soln1, graph))
  # ucs_num_nodes.append(total_nodes)
  # ucs_realtime.append(elapsed_time)
  # ucs_cpu_time.append(elapsed_process_time)

  # start_time = time.time()
  # start_process_time = time.process_time()
  # soln2 = astar_search(prob, random_edges_heuristic).solution()[size - 1]
  # elapsed_process_time = time.process_time() - start_process_time
  # elapsed_time = time.time() - start_time

  # rand_cost.append(calc_cost(soln2, graph))
  # rand_num_nodes.append(total_nodes)
  # rand_realtime.append(elapsed_time)
  # rand_cpu_time.append(elapsed_process_time)

  # start_time = time.time()
  # start_process_time = time.process_time()
  # soln3 = astar_search(prob, cheapest_edges_heuristic).solution()[size - 1]
  # elapsed_process_time = time.process_time() - start_process_time
  # elapsed_time = time.time() - start_time

  # cheapest_cost.append(calc_cost(soln3, graph))
  # cheapest_num_nodes.append(total_nodes)
  # cheapest_realtime.append(elapsed_time)
  # cheapest_cpu_time.append(elapsed_process_time)

  start_time = time.time()
  start_process_time = time.process_time()
  soln4 = astar_search(prob, mst_heuristic).solution()[size - 1]
  elapsed_process_time = time.process_time() - start_process_time
  elapsed_time = time.time() - start_time

  mst_cost.append(calc_cost(soln4, graph))
  mst_num_nodes.append(total_nodes)
  mst_realtime.append(elapsed_time)
  mst_cpu_time.append(elapsed_process_time)

  

  start_time = time.time()
  start_process_time = time.process_time()
  g=GeneticAlgorithm(size, graphMatrix, dict_numbers_to_letters)
  genetic_initial_pop=g.init_population(100,numberOfVertices)
  genetic_fitness=g.fitness_fn
  gene_pool=[]
  for x in range(numberOfVertices):
    gene_pool.append(dict_numbers_to_letters[x])
  sol=g.genetic_algorithm(genetic_initial_pop,genetic_fitness,gene_pool, 100)
  gensol=sol
  gensol.append(gensol[0])
  elapsed_process_time = time.process_time() - start_process_time
  elapsed_time = time.time() - start_time

  ga_cost.append(calc_cost(gensol, graph))
  ga_num_nodes.append(total_nodes)
  ga_realtime.append(elapsed_time)
  ga_cpu_time.append(elapsed_process_time)
    
  all_cities.sort()
  distances = graphDict

  tsp = TSP_problem(all_cities, distances)

  start_time = time.time()
  start_process_time = time.process_time()
  soln6 = hill_climbing(tsp)
  soln6.append(soln6[0])
  elapsed_process_time = time.process_time() - start_process_time
  elapsed_time = time.time() - start_time

  hc_cost.append(calc_cost(soln6, graph))
  hc_num_nodes.append(total_nodes)
  hc_realtime.append(elapsed_time)
  hc_cpu_time.append(elapsed_process_time)

  start_time = time.time()
  start_process_time = time.process_time()
  soln7 = simulated_annealing(tsp)
  soln7.append(soln7[0])
  elapsed_process_time = time.process_time() - start_process_time
  elapsed_time = time.time() - start_time

  sa_cost.append(calc_cost(soln7, graph))
  sa_num_nodes.append(total_nodes)
  sa_realtime.append(elapsed_time)
  sa_cpu_time.append(elapsed_process_time)


# ucs_stats.append([size,  min(ucs_cost), sum(ucs_cost) / len(ucs_cost), max(ucs_cost),
#                      min(ucs_num_nodes), sum(ucs_num_nodes) / len(ucs_num_nodes), max(ucs_num_nodes),
#                      min(ucs_realtime), sum(ucs_realtime) / len(ucs_realtime), max(ucs_realtime),
#                      min(ucs_cpu_time), sum(ucs_cpu_time) / len(ucs_cpu_time), max(ucs_cpu_time)])

# cheapest_edges_stats.append([size, min(cheapest_cost), sum(cheapest_cost) / len(cheapest_cost), max(cheapest_cost),
#                       min(cheapest_num_nodes), sum(cheapest_num_nodes) / len(cheapest_num_nodes), max(cheapest_num_nodes),
#                       min(cheapest_realtime), sum(cheapest_realtime) / len(cheapest_realtime), max(cheapest_realtime),
#                       min(cheapest_cpu_time), sum(cheapest_cpu_time) / len(cheapest_cpu_time), max(cheapest_cpu_time)])

# random_edges_stats.append([size, min(rand_cost), sum(rand_cost) / len(rand_cost), max(rand_cost),
#                       min(rand_num_nodes), sum(rand_num_nodes) / len(rand_num_nodes), max(rand_num_nodes),
#                       min(rand_realtime), sum(rand_realtime) / len(rand_realtime), max(rand_realtime),
#                       min(rand_cpu_time), sum(rand_cpu_time) / len(rand_cpu_time), max(rand_cpu_time)])

mst_stats.append([size, min(mst_cost), sum(mst_cost) / len(mst_cost), max(mst_cost),
                      min(mst_num_nodes), sum(mst_num_nodes) / len(mst_num_nodes), max(mst_num_nodes),
                      min(mst_realtime), sum(mst_realtime) / len(mst_realtime), max(mst_realtime),
                      min(mst_cpu_time), sum(mst_cpu_time) / len(mst_cpu_time), max(mst_cpu_time)])

hc_stats.append([size, min(hc_cost), sum(hc_cost) / len(hc_cost), max(hc_cost),
                      min(hc_num_nodes), sum(hc_num_nodes) / len(hc_num_nodes), max(hc_num_nodes),
                      min(hc_realtime), sum(hc_realtime) / len(hc_realtime), max(hc_realtime),
                      min(hc_cpu_time), sum(hc_cpu_time) / len(hc_cpu_time), max(hc_cpu_time)])

sa_stats.append([size, min(sa_cost), sum(sa_cost) / len(sa_cost), max(sa_cost),
                      min(sa_num_nodes), sum(sa_num_nodes) / len(sa_num_nodes), max(sa_num_nodes),
                      min(sa_realtime), sum(sa_realtime) / len(sa_realtime), max(sa_realtime),
                      min(sa_cpu_time), sum(sa_cpu_time) / len(sa_cpu_time), max(sa_cpu_time)])
  
ga_stats.append([size, min(ga_cost), sum(ga_cost) / len(ga_cost), max(ga_cost),
                      min(ga_num_nodes), sum(ga_num_nodes) / len(ga_num_nodes), max(ga_num_nodes),
                      min(ga_realtime), sum(ga_realtime) / len(ga_realtime), max(ga_realtime),
                      min(ga_cpu_time), sum(ga_cpu_time) / len(ga_cpu_time), max(ga_cpu_time)])


final =[]
final.append(ucs_stats)
final.append(cheapest_edges_stats)
final.append(random_edges_stats)
final.append(mst_stats)

from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/PT3_2.json', 'a') as f:
    #f.write("%s\n" % sum(ucs_stats, []))
    #f.write("%s\n" % sum(cheapest_edges_stats, []))
    # f.write("%s\n" % sum(random_edges_stats, []))
    f.write("%s\n" % sum(mst_stats, []))
    f.write("%s\n" % sum(hc_stats, []))
    f.write("%s\n" % sum(sa_stats, []))
    f.write("%s\n" % sum(ga_stats, []))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/Sandbox_1.json')
df.columns=fields
df

,Size,Min Cost,Avg Cost,Max Cost,Min Nodes Expanded,Avg Nodes Expanded,Max Nodes Expanded,Min Real Time,Avg Real Time,Max Real Time,Min CPU Time,Avg CPU Time,Max CPU Time
0,5,1135.0,2274.966667,2933.0,45,52.600000,64,0.000840,0.001001,0.001374,0.000838,0.000998,0.001369
1,5,1135.0,2274.966667,2933.0,37,48.966667,64,0.001072,0.001280,0.001565,0.001070,0.001278,0.001563
2,5,1135.0,2292.033333,2960.0,13,24.033333,34,0.000366,0.000726,0.001000,0.000364,0.000724,0.000998
3,5,1135.0,2274.966667,2933.0,41,52.033333,64,0.002717,0.003440,0.004515,0.002714,0.003436,0.004509
4,7,2126.0,2701.266667,3349.0,543,788.800000,1100,0.123613,0.207088,0.327921,0.123495,0.206520,0.326018
5,7,2126.0,2701.266667,3349.0,259,483.200000,868,0.039423,0.095066,0.188668,0.039412,0.094733,0.188471
6,7,2126.0,2793.700000,3582.0,17,82.166667,221,0.001064,0.008131,0.026634,0.001062,0.008111,0.026623
7,7,2126.0,2701.266667,3349.0,218,546.366667,915,0.055547,0.166878,0.350661,0.055459,0.166458,0.349045
